#Install The Dependencies

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.1 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"]= userdata.get('GROQ_API_KEY')

In [3]:
from groq import Groq
client = Groq()

In [4]:
!pip install python-dotenv

#To Generate the Topic Points


In [5]:
from groq import Groq
import os
from dotenv import load_dotenv

# Load environment variables from .env.local file
load_dotenv('.env.local')

# Initialize Groq client
client = Groq(
    api_key=os.getenv('GROQ_API_KEY')
)

class RevisionManager:
    def __init__(self):
        self._revision = None

    def set_revision(self, data):
        self._revision = data

    def get_revision(self):
        return self._revision

# Initialize revision manager
revision = RevisionManager()

def generate_guide(selected_subject, selected_qualification, additional_context):
    """
    Generate revision guide using Groq API
    """
    try:
        ai_response = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=[
                {
                    "role": "system",
                    "content": f"Generate an array of guide sentences like formulas or ideas for the {selected_subject} subject for {selected_qualification}. Additional context: {additional_context}. Give it to me in form of array of points and it must contain 5-8 points. Only provide me array don't provide me any other informations."
                }
            ]
        )

        # Store the response in revision manager
        revision.set_revision(ai_response.choices[0].message.content)

        return {
            "message": "Revision data updated successfully",
            "revision": revision.get_revision()
        }

    except Exception as e:
        print(f"Error generating guide: {str(e)}")
        return {"error": "Internal Server Error"}

def get_guide():
    """
    Retrieve the stored revision guide
    """
    return {
        "revision": revision.get_revision()
    }



#Enter the Input Values (e.g subject, qualifications, additional details)

In [6]:
if __name__ == "__main__":
    subject = input("Enter the subject: ")
    qualification = input("Enter the qualification: ")
    context = input("Enter additional context: ")



Enter the subject: physics
Enter the qualification: 10
Enter additional context: thermodynamics 


#Get The Topics related to the Subjects

In [7]:
result = generate_guide(subject, qualification, context)
print("\nGenerated Guide:")

for i, line in enumerate(result['revision'].split('\n'), start=1):
    print(f"{i}. {line}")
print("\n")

stored_guide = get_guide()


# for i, line in enumerate(stored_guide['revision'].split('\n'), start=1):
#     print(f"{i}. {line}")
# print("\n")


Generated Guide:
1. [
2. "1. Thermodynamics deals with the relationship between heat, work, and energy in a system.",
3. "2. The four laws of thermodynamics provide a fundamental framework for understanding the behavior of physical systems.",
4. "3. The first law, also known as the law of conservation of energy, states that energy cannot be created or destroyed, only transformed.",
5. "4. The second law introduces the concept of entropy, a measure of system's disorder or randomness that tends to increase over time.",
6. "5. The third law defines the absolute zero temperature, a state where entropy reaches its minimum value.",
7. "6. The fourth law establishes the relationship between the entropy of a system and its reversibility or irreversibility.",
8. "7. Thermodynamic processes, such as adiabatic, isothermal, isobaric, and isochoric, are described by the changes in system's properties during heat transfer or work.",
9. "8. Real-world applications include engines, refrigeration, and

#To Generate Quizs on the Topic

In [8]:
import os
import requests

# Set up the Groq API base URL and API key
GROQ_API_BASE_URL = "https://api.groq.com/openai/v1/chat/completions"
API_KEY = os.environ.get("GROQ_API_KEY")  # Ensure the API key is set in your environment

def generate_quiz(topic, num_questions=5):
    """
    Generate quiz questions using the mixtral-8x7b-32768 model via Groq API.

    :param topic: The topic for which to generate quiz questions.
    :param num_questions: Number of questions to generate.
    :return: List of generated quiz questions.
    """
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Prompt for the AI model
    prompt = (
        f"Generate {num_questions} quiz questions on the topic: {topic}. "
        "Each question should be concise and relevant to the topic."
    )

    payload = {
        "model": "llama-3.3-70b-versatile",
        "messages": [
            {"role": "system", "content": "You are a quiz question generator."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens":1024,
        "top_p": 1
    }

    try:
        # Make the POST request to the Groq API
        response = requests.post(GROQ_API_BASE_URL, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the response JSON
        result = response.json()

        # Extract generated quiz questions
        generated_content = result["choices"][0]["message"]["content"]
        quiz_questions = [question.strip() for question in generated_content.split("\n") if question.strip()]

        return quiz_questions

    except requests.exceptions.RequestException as e:
        print(f"Error connecting to Groq API: {e}")
        return []
    except KeyError as e:
        print(f"Error parsing response: {e}")
        return []

def main():
    print("Welcome to the Quiz Generator!\n")

    # Take topic input from the user
    topic = input("Enter the topic for the quiz: ").strip()
    num_questions = int(input("How many quiz questions would you like to generate? (default is 5): ") or 5)

    # Generate quiz questions
    print("\nGenerating quiz questions...")
    quiz_questions = generate_quiz(topic, num_questions)

    if quiz_questions:
        print("\nGenerated Quiz Questions:")
        for i, question in enumerate(quiz_questions, start=1):
            print(f"{i}. {question}")
    else:
        print("\nFailed to generate quiz questions. Please try again later.")

if __name__ == "__main__":
    main()


Welcome to the Quiz Generator!

Enter the topic for the quiz: The first law, also known as the law of conservation of energy, states that energy cannot be created or destroyed, only transformed.
How many quiz questions would you like to generate? (default is 5): 4

Generating quiz questions...

Generated Quiz Questions:
1. Here are 4 quiz questions on the topic:
2. 1. What is the fundamental principle of the first law of energy?
3. A) Energy can be created
4. B) Energy cannot be created or destroyed
5. C) Energy can only be destroyed
6. D) Energy can only be created
7. Answer: B) Energy cannot be created or destroyed
8. 2. According to the law of conservation of energy, what happens to energy?
9. A) It is lost over time
10. B) It is gained over time
11. C) It is transformed from one form to another
12. D) It remains constant but never changes form
13. Answer: C) It is transformed from one form to another
14. 3. Which of the following is an example of energy transformation?
15. A) Creat